# Agent Types

Toda a ideia de utilizarmos agente é termos um sistema que saiba que possue ferramntas diferentes em mãos para utilizar e aumentar suas capacidades e as utilize nos momentos adequados. Para que isso seja possível, a OpenAI fez diversos processos de fine tunning em seus modelos, de forma que eles consiga gerar outputs paronizadas para a utilização das ferramentas. Entretanto, muito desse processo pode ser feito, ou melhorado, através de Engenharia de Prompts, e por isso, diversos tipos de agentes (Agent Types) foram criados na literatura e no framework LangChain. Cada um tenta otimizar a utilização das ferramentas e vamos falar sobre os dois mais utilizados: Toll Calling e ReAct. Para verificar todos, você pode acessar a página: https://python.langchain.com/v0.1/docs/modules/agents/agent_types/.

## Tool calling agent

Tool calling permite que um modelo detecte quando uma ou mais ferramentas devem ser acionadas e responda com os dados que devem ser passados para essas ferramentas. Em uma chamada de API, você pode descrever ferramentas e fazer com que o modelo escolha inteligentemente gerar um objeto estruturado, como JSON, contendo argumentos para acionar essas ferramentas. O objetivo das APIs de ferramentas é retornar chamadas de ferramentas válidas e úteis de forma mais confiável do que o que pode ser feito usando uma API genérica de conclusão de texto ou chat.

Podemos aproveitar essa saída estruturada, combinada com o fato de que você pode vincular várias ferramentas a um modelo de chat que chama ferramentas e permitir que o modelo escolha qual chamar, para criar um agente que repetidamente chama ferramentas e recebe resultados até que uma consulta seja resolvida.

Esta é uma versão mais generalizada do agente de ferramentas do OpenAI, que foi projetada para o estilo específico de chamada de ferramentas da OpenAI. Ela usa a interface ToolCall do LangChain para suportar uma gama mais ampla de implementações de provedores, como Anthropic, Google Gemini e Mistral, além do OpenAI.

In [2]:
from langchain_experimental.tools import PythonAstREPLTool

tools = [PythonAstREPLTool()]

In [3]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

system_msg = '''Você é um assistente amigável.
Certifique-se de usar a ferramenta PythonAstREPLTool para auxiliar a responder as perguntas'''

prompt = ChatPromptTemplate.from_messages([
    ('system', system_msg),
    ('placeholder', '{chat_history}'),
    ('human', '{input}'),
    ('placeholder', '{agent_scratchpad}')
])

chat = ChatOpenAI()
agent = create_tool_calling_agent(chat, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [4]:
agent_executor.invoke({'input': 'Qual é o décimo valor da sequência finonacci?'})



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'def fibonacci(n):\n    if n <= 1:\n        return n\n    else:\n        return fibonacci(n-1) + fibonacci(n-2)\n\nfibonacci(10)'}`


NameError: name 'fibonacci' is not defined
Invoking: `python_repl_ast` with `{'query': 'def fibonacci(n):\n    if n <= 1:\n        return n\n    else:\n        return fibonacci(n-1) + fibonacci(n-2)\n\nfibonacci(9)'}`
responded: Parece que houve um erro na definição da função fibonacci. Vou corrigir isso e tentar novamente.

NameError: name 'fibonacci' is not defined
Invoking: `python_repl_ast` with `{'query': 'def fibonacci(n):\n    if n <= 1:\n        return n\n    else:\n        return fibonacci(n-1) + fibonacci(n-2)\n\nfibonacci(10)'}`
responded: Peço desculpas pela confusão. Vou corrigir a chamada da função fibonacci para obter o décimo valor da sequência Fibonacci. Vamos tentar novamente.

NameError: name 'fibonacci' is not defined
Invoking: `python_repl_ast` with 

{'input': 'Qual é o décimo valor da sequência finonacci?',
 'output': 'O décimo valor da sequência Fibonacci é 34.'}

In [5]:
agent_executor.invoke({'input': 'Quantas letras tem a palavra LangChain?'})



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "len('LangChain')"}`


9A palavra "LangChain" tem 9 letras.

> Finished chain.


{'input': 'Quantas letras tem a palavra LangChain?',
 'output': 'A palavra "LangChain" tem 9 letras.'}

In [6]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

system_msg = """Você é um agente projetado para escrever e executar código Python para responder perguntas.
Você tem acesso a um REPL Python, que pode usar para executar código Python.
Se encontrar um erro, depure o código e tente novamente.
Use apenas a saída do seu código para responder à pergunta.
Você pode conhecer a resposta sem executar nenhum código, mas deve ainda assim executar o código para obter a resposta.
Se não parecer possível escrever código para responder à pergunta, simplesmente retorne "Não sei" como a resposta."""


prompt = ChatPromptTemplate.from_messages([
    ('system', system_msg),
    ('placeholder', '{chat_history}'),
    ('human', '{input}'),
    ('placeholder', '{agent_scratchpad}')
])

chat = ChatOpenAI()
agent = create_tool_calling_agent(chat, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [7]:
agent_executor.invoke({'input': 'Quantas letras tem a palavra LangChain?'})



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "len('LangChain')"}`


9A palavra "LangChain" tem 9 letras.

> Finished chain.


{'input': 'Quantas letras tem a palavra LangChain?',
 'output': 'A palavra "LangChain" tem 9 letras.'}

## ReAct agent (Reason + Act)

É uma técnica de Engenharia de Promots criada para permitir que os LLMs interajam com ferramentas externas para recuperar informações adicionais que levam a respostas mais confiáveis e factuais.


In [8]:
from langchain.agents import create_react_agent

https://smith.langchain.com/hub/

In [9]:
from langchain import hub

prompt = hub.pull('hwchase17/react')
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [12]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [13]:
chat = ChatOpenAI()
agent = create_react_agent(chat, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [14]:
agent_executor.invoke({'input': 'Qual é o décimo valor da sequência fibonacci?'})



> Entering new AgentExecutor chain...
I can use Python to generate the Fibonacci sequence and then extract the tenth value from it.
Action: [python_repl_ast]
Action Input: fib = lambda n: n if n <= 1 else fib(n-1) + fib(n-2)[python_repl_ast] is not a valid tool, try one of [python_repl_ast].I can't use [python_repl_ast], I will try again.
Action: [python_repl_ast]
Action Input: fib = lambda n: n if n <= 1 else fib(n-1) + fib(n-2)[python_repl_ast] is not a valid tool, try one of [python_repl_ast].I will use the Python shell directly to generate the Fibonacci sequence and extract the tenth value.
Action: [python_repl_ast]
Action Input: fib = lambda n: n if n <= 1 else fib(n-1) + fib(n-2)[python_repl_ast] is not a valid tool, try one of [python_repl_ast].I will try using the Python shell directly without the tool.
Action: [python_repl_ast]
Action Input: fib = lambda n: n if n <= 1 else fib(n-1) + fib(n-2)[python_repl_ast] is not a valid tool, try one of [python_repl_ast].I will use the 

{'input': 'Qual é o décimo valor da sequência fibonacci?',
 'output': 'The tenth value of the Fibonacci sequence is 55.'}

In [19]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
'''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of {tool_names}
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''
)

chat = ChatOpenAI()
agent = create_react_agent(chat, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [16]:
agent_executor.invoke({'input': 'Qual é o décimo valor da sequência fibonacci?'})



> Entering new AgentExecutor chain...
I can use Python to generate the Fibonacci sequence and then find the 10th value.
Action: python_repl_ast
Action Input: fib = [0, 1]Now I need to generate the Fibonacci sequence up to the 10th value.
Action: python_repl_ast
Action Input: for i in range(2, 10):IndentationError: expected an indented block after 'for' statement on line 1 (<unknown>, line 1)I need to properly indent the for loop.
Action: python_repl_ast
Action Input: for i in range(2, 10):IndentationError: expected an indented block after 'for' statement on line 1 (<unknown>, line 1)I need to include the code to update the Fibonacci sequence within the for loop.
Action: python_repl_ast
Action Input: for i in range(2, 10):IndentationError: expected an indented block after 'for' statement on line 1 (<unknown>, line 1)I need to make sure I properly indent the code block within the for loop.
Action: python_repl_ast
Action Input: fib.append(fib[-1] + fib[-2])I should now print out the 10t

{'input': 'Qual é o décimo valor da sequência fibonacci?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [22]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
'''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of {tool_names}
Action Input: the input to the action.
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''
)

chat = ChatOpenAI(model="gpt-3.5-turbo-0125")
agent = create_react_agent(chat, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({'input': 'Qual é o décimo valor da sequência fibonacci?'})



> Entering new AgentExecutor chain...
We can use a Python program to generate the Fibonacci sequence and then find the tenth value.
Action: python_repl_ast
Action Input: def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

fibonacci(10)NameError: name 'fibonacci' is not definedWe need to define the fibonacci function before we can call it.
Action: python_repl_ast
Action Input: def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

fibonacci(10)NameError: name 'fibonacci' is not definedWe need to call the fibonacci function with the correct input parameter.
Action: python_repl_ast
Action Input: fibonacci(9)NameError: name 'fibonacci' is not definedWe need to define the fibonacci function and then call it with the correct input parameter.
Action: python_repl_ast
Action Input: def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2

{'input': 'Qual é o décimo valor da sequência fibonacci?', 'output': '55'}